In [0]:
import torch
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from collections import Counter

In [0]:
categories = ["comp.graphics","sci.space","rec.sport.baseball"]
news_group_train = fetch_20newsgroups(subset = 'train', categories = categories)
news_group_test = fetch_20newsgroups(subset = 'test', categories = categories) 

In [0]:
 news_group_train.target

In [0]:
len(news_group_train.data),len(news_group_test.data)

In [0]:
vocab = Counter()

for text in news_group_train.data:
  for word in text.split(' '):
    vocab[word.lower()] += 1

for text in news_group_test.data:
  for word in text.split(' '):
    vocab[word.lower()] += 1
    
total_words = len(vocab) 

In [0]:
total_words

In [0]:
def get_word_2_index(vocab):
  word2index = {}
  for i,word in enumerate(vocab):
    word2index[word.lower()] = i
  return word2index

word2index = get_word_2_index(vocab)
  

In [0]:
len(word2index)

In [0]:
def get_batches(df, i, batch_size):
  batches = []
  results = []
  
  texts = df.data[i*batch_size: i*batch_size+ batch_size]
  categories = df.target[i*batch_size:i*batch_size+batch_size]
  
  for text in texts:
    layer = np.zeros(total_words,dtype = float)
    for word in text.split(' '):
        layer[word2index[word.lower()]] += 1
     
    batches.append(layer)
    
  for category in categories:
    index_ = -1
    if category == 0:
      index_y = 0
    elif category == 1:
      index_y = 1
    else:
      index_y = 2
    results.append(index_y)
    
  return np.array(batches), np.array(results)

In [0]:
learning_rate = 0.01
num_epochs = 10
batch_size = 150
display_step = 1


hidden_size = 100
input_size = total_words
num_classes = 3


In [0]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class OurNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(OurNet,self).__init__()
    self.layer_1 = nn.Linear(input_size,hidden_size, bias = True)
    self.relu = nn.ReLU()
    self.layer_2 = nn.Linear(hidden_size,hidden_size, bias = True)
    self.output_layer = nn.Linear(hidden_size, num_classes, bias = True)
    
  def forward(self,x):
    out = self.layer_1(x)
    out = self.relu(out)
    out = self.layer_2(out)
    out = self.relu(out)
    out = self.output_layer(out)
    return out

In [0]:
net = OurNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)

In [0]:
loss = nn.CrossEntropyLoss()
input = Variable(torch.randn(2, 5), requires_grad=True)
print(">>> batch of size 2 and 5 possible classes")
print(input)
target = Variable(torch.LongTensor(2).random_(5))
print(">>> array of size 'batch_size' with the index of the maxium label for each item")
print(target)
output = loss(input, target)
output.backward()

In [0]:
for epoch in range(num_epochs):
    total_batch = int(len(news_group_train.data)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batches(news_group_train,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        #print("articles",articles)
        #print(batch_x, labels)
        #print("size labels",labels.size())
        
        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        if (i+1) % 4 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, i+1, len(news_group_train.data)//batch_size, loss.data))

In [0]:
# Test the Model
correct = 0
total = 0
total_test_data = len(news_group_test.target)
batch_x_test,batch_y_test = get_batches(news_group_test,0,total_test_data)
articles = Variable(torch.FloatTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)
outputs = net(articles)
_, predicted = torch.max(outputs.data, 1)
total += labels.size(0)
correct += (predicted == labels).sum()
    
print('Accuracy of the network on the 1180 test articles: %d %%' % (100 * correct / total))